In [3]:
from glove import Corpus, Glove
import pandas as pd
import logging


questions = pd.read_csv('data/2018-12-10-Treatd/LemmatizedQAFix.csv', sep=';', header=0)
terms = pd.read_csv('data/2018-12-10-Treatd/LemmatizedTerms.csv', sep=";")
termset = set(terms['term'])
termset.add('leite')
termset.add('boi')
termset.add('carrapato')

logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s', level=logging.INFO)




In [4]:
def corpusreader(text):
    for i, row in text.iterrows():
        print(row['QuestionTerms'])
        if(i%10 == 0):
            logging.info("Read {0} reviews".format(i))
        mylist = row['QuestionTerms'].split()
        mylist.extend(row['AnswerTerms'].split())
        yield [w for w in mylist if w in termset]

documents = list(corpusreader(questions))
print(documents)
logging.info("Done reading data file")

2018-12-18 11:11:12,141: INFO : Read 0 reviews
2018-12-18 11:11:12,147: INFO : Read 10 reviews
2018-12-18 11:11:12,149: INFO : Read 20 reviews
2018-12-18 11:11:12,153: INFO : Read 30 reviews
2018-12-18 11:11:12,156: INFO : Read 40 reviews
2018-12-18 11:11:12,158: INFO : Read 50 reviews
2018-12-18 11:11:12,160: INFO : Read 60 reviews
2018-12-18 11:11:12,163: INFO : Read 70 reviews
2018-12-18 11:11:12,165: INFO : Read 80 reviews
2018-12-18 11:11:12,169: INFO : Read 90 reviews
2018-12-18 11:11:12,173: INFO : Read 100 reviews
2018-12-18 11:11:12,175: INFO : Read 110 reviews
2018-12-18 11:11:12,184: INFO : Read 120 reviews
2018-12-18 11:11:12,186: INFO : Read 130 reviews
2018-12-18 11:11:12,189: INFO : Read 140 reviews
2018-12-18 11:11:12,192: INFO : Read 150 reviews
2018-12-18 11:11:12,194: INFO : Read 160 reviews
2018-12-18 11:11:12,197: INFO : Read 170 reviews
2018-12-18 11:11:12,199: INFO : Read 180 reviews
2018-12-18 11:11:12,201: INFO : Read 190 reviews
2018-12-18 11:11:12,204: INFO :

1 quando iniciar o cuidado com o bezerro ? 
2 qual o principal cuidado que se dever tomar com o bezerro recém-nascer ? 
3 como se explicar o nascimento de bezerro fraco e pequeno ? 
4 qual o bom manejo de bezerro recém-nascer a fim de evitar o proliferação de doença ? 
5 qual o bom maneira de criar o bezerro : ao pé dar mãe ou apartar dela ? 
6 ser correto deixar para o bezerro apenas a “rapa de leite do quatro tetos” ( leite residual após o ordenha ) ? 
7 qual o vantagem do aleitamento natural ? qual o vantagem dar aleitamento artificial ? 
8 por que algum produtor fracassam ao mudar dar sistema de aleitamento natural para o artificial ? 
9 qual qualidade dever ter o pessoa que tratar do bezerro ? 
10 qual o causa dar mortalidade de bezerro no 3 primeiro mês de vida ? 
11 qual o vantagem de descornar o bezerro ainda jovem ? 
12 o leite “sujo” ser importante para o bezerro ? 
13 o colostro ser diferente do leitar normal ? seu composição variar após o parto ? 
14 haver diferença na prod

In [13]:
corpus = Corpus()
corpus.fit(documents, window=10)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=100, no_threads=4, verbose=False)

In [15]:
glove.add_dictionary(corpus.dictionary)

In [16]:
relations = pd.DataFrame()
for term in termset:
    if term in glove.dictionary:
        for otherTerm in glove.most_similar(term):
            relations = relations.append({'source':term, 'target':otherTerm[0]}, ignore_index=True)
            #print(term, otherTerm[0])

In [17]:
relations.to_csv('data/2018-12-18-Treatd/gloverelations100epoch.csv')